In [36]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', None) 
import numpy as np
import statistics
import math

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.tree import export_graphviz, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from statsmodels.api import add_constant
import statsmodels.formula.api as smf
from statsmodels.api import Logit

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

In [37]:
# 글씨체 분석
matplotlib.rc("font", family = "NanumGothic")
matplotlib.rc("axes", unicode_minus = False )

In [38]:
df = pd.read_csv('B3_반도체/반도체_생산실적_현황.csv')

In [39]:
print(df.shape)
df.head()

(1704, 59)


,No_Die,Ox_Chamber,process,type,Temp_OXid,Vapor,ppm,Pressure,Oxid_time,thickness,Lot_Num,Wafer_Num,Datetime,photo_soft_Chamber,process 2,resist_target,N2_HMDS,pressure_HMDS,temp_HMDS,temp_HMDS_bake,time_HMDS_bake,spin1,spin2,spin3,photoresist_bake,temp_softbake,time_softbake,lithography_Chamber,Process 2-1,Line_CD,UV_type,Wavelength,Resolution,Energy_Exposure,Etching_Chamber,Process 3,Thin F4,Thin F3,Thin F2,Thin F1,Temp_Etching,Source_Power,Selectivity,Chamber_Num,process4,Flux60s,Flux90s,Flux160s,Flux480s,Flux840s,input_Energy,Temp_implantation,Furance_Temp,RTA_Temp,Target,Error_message,Wafer_map,path,Defect
0,NOLSM325131325,2,Oxidation,dry,1214.307948,O2,26.69,0.203,120,713.132,13,28,01-03-2022,1,Photo,1.46994,17.367,15.006,20.001,200.551,90.001,502.391,4017.144,4903.611,5.020,91.918,30.005,1,photo_li,41.676,I,365,505.388,109.628,1,Etching,340.0,1522.0,3644.0,5732.0,72.651,52.103,1.030,1,Implantation,1.500000e+16,1.320000e+17,6.470000e+17,3.010000e+17,6.000000e+17,30795.934,103.433,854.0,154,141.0,none,[[0 0 0 0 0 0 0 0 0 0 1 2 1 1 1 2 0 0 0 0 0 0 ...,2111,0
1,NOLSM326132326,2,Oxidation,dry,978.110792,O2,30.86,0.217,137,714.295,13,29,01-03-2022,1,Photo,0.86579,16.090,15.059,19.993,199.835,89.999,507.183,4072.913,4979.082,4.937,90.976,30.005,1,photo_li,53.612,H,405,541.732,104.917,2,Etching,265.0,1513.0,3631.0,5729.0,70.267,52.004,1.038,2,Implantation,1.110000e+16,4.370000e+16,1.040000e+18,3.030000e+17,6.000000e+17,32135.618,105.552,895.0,156,55.0,none,[[0 0 0 0 0 0 0 0 0 0 2 2 2 2 1 2 0 0 0 0 0 0 ...,2112,0
2,NOLSM327133327,2,Oxidation,dry,1176.019149,O2,31.07,0.218,116,710.114,13,30,01-03-2022,1,Photo,0.90483,19.026,14.930,19.995,202.118,90.001,501.286,4084.939,5031.989,4.888,91.969,30.006,1,photo_li,47.598,I,365,532.200,106.632,3,Etching,411.0,1568.0,3653.0,5729.0,71.231,50.846,1.005,3,Implantation,1.040000e+16,1.510000e+16,6.470000e+17,2.980000e+17,6.000000e+17,31057.895,102.598,898.0,152,96.0,none,[[0 0 0 0 0 0 0 0 0 0 1 2 1 1 1 2 0 0 0 0 0 0 ...,2113,0
3,NOLSM328134328,2,Oxidation,dry,933.442733,O2,31.04,0.196,143,710.617,13,31,01-03-2022,1,Photo,0.42197,18.891,14.947,20.004,194.668,90.000,503.396,4045.078,4969.316,4.933,91.458,30.001,1,photo_li,32.843,G,436,537.729,108.785,1,Etching,328.0,1326.0,3661.0,5718.0,71.295,51.665,1.025,1,Implantation,8.885140e+15,1.020000e+17,3.410000e+17,3.000000e+17,6.000000e+17,32140.566,102.963,879.0,155,105.0,none,[[0 0 0 0 0 0 0 0 0 0 2 1 1 1 1 2 0 0 0 0 0 0 ...,2111,0
4,NOLSM329135329,2,Oxidation,wet,1140.598752,H2O,31.37,0.215,76,711.632,13,32,01-03-2022,1,Photo,1.70139,13.788,14.820,19.998,202.192,90.000,501.423,4010.176,5092.407,5.050,94.723,29.988,1,photo_li,57.370,H,405,522.143,109.913,2,Etching,219.0,1451.0,3637.0,5729.0,72.983,50.787,1.038,2,Implantation,1.670000e+16,7.880000e+16,7.260000e+17,3.020000e+17,6.000000e+17,31986.009,101.419,882.0,155,79.0,none,[[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 ...,2112,0


In [40]:
# 결측치 제거
df.dropna(inplace = True)
df

,No_Die,Ox_Chamber,process,type,Temp_OXid,Vapor,ppm,Pressure,Oxid_time,thickness,Lot_Num,Wafer_Num,Datetime,photo_soft_Chamber,process 2,resist_target,N2_HMDS,pressure_HMDS,temp_HMDS,temp_HMDS_bake,time_HMDS_bake,spin1,spin2,spin3,photoresist_bake,temp_softbake,time_softbake,lithography_Chamber,Process 2-1,Line_CD,UV_type,Wavelength,Resolution,Energy_Exposure,Etching_Chamber,Process 3,Thin F4,Thin F3,Thin F2,Thin F1,Temp_Etching,Source_Power,Selectivity,Chamber_Num,process4,Flux60s,Flux90s,Flux160s,Flux480s,Flux840s,input_Energy,Temp_implantation,Furance_Temp,RTA_Temp,Target,Error_message,Wafer_map,path,Defect
0,NOLSM325131325,2,Oxidation,dry,1214.307948,O2,26.69,0.203,120,713.132,13,28,01-03-2022,1,Photo,1.46994,17.367,15.006,20.001,200.551,90.001,502.391,4017.144,4903.611,5.020,91.918,30.005,1,photo_li,41.676,I,365,505.388,109.628,1,Etching,340.0,1522.0,3644.0,5732.0,72.651,52.103,1.030,1,Implantation,1.500000e+16,1.320000e+17,6.470000e+17,3.010000e+17,6.000000e+17,30795.934,103.433,854.0,154,141.0,none,[[0 0 0 0 0 0 0 0 0 0 1 2 1 1 1 2 0 0 0 0 0 0 ...,2111,0
1,NOLSM326132326,2,Oxidation,dry,978.110792,O2,30.86,0.217,137,714.295,13,29,01-03-2022,1,Photo,0.86579,16.090,15.059,19.993,199.835,89.999,507.183,4072.913,4979.082,4.937,90.976,30.005,1,photo_li,53.612,H,405,541.732,104.917,2,Etching,265.0,1513.0,3631.0,5729.0,70.267,52.004,1.038,2,Implantation,1.110000e+16,4.370000e+16,1.040000e+18,3.030000e+17,6.000000e+17,32135.618,105.552,895.0,156,55.0,none,[[0 0 0 0 0 0 0 0 0 0 2 2 2 2 1 2 0 0 0 0 0 0 ...,2112,0
2,NOLSM327133327,2,Oxidation,dry,1176.019149,O2,31.07,0.218,116,710.114,13,30,01-03-2022,1,Photo,0.90483,19.026,14.930,19.995,202.118,90.001,501.286,4084.939,5031.989,4.888,91.969,30.006,1,photo_li,47.598,I,365,532.200,106.632,3,Etching,411.0,1568.0,3653.0,5729.0,71.231,50.846,1.005,3,Implantation,1.040000e+16,1.510000e+16,6.470000e+17,2.980000e+17,6.000000e+17,31057.895,102.598,898.0,152,96.0,none,[[0 0 0 0 0 0 0 0 0 0 1 2 1 1 1 2 0 0 0 0 0 0 ...,2113,0
3,NOLSM328134328,2,Oxidation,dry,933.442733,O2,31.04,0.196,143,710.617,13,31,01-03-2022,1,Photo,0.42197,18.891,14.947,20.004,194.668,90.000,503.396,4045.078,4969.316,4.933,91.458,30.001,1,photo_li,32.843,G,436,537.729,108.785,1,Etching,328.0,1326.0,3661.0,5718.0,71.295,51.665,1.025,1,Implantation,8.885140e+15,1.020000e+17,3.410000e+17,3.000000e+17,6.000000e+17,32140.566,102.963,879.0,155,105.0,none,[[0 0 0 0 0 0 0 0 0 0 2 1 1 1 1 2 0 0 0 0 0 0 ...,2111,0
4,NOLSM329135329,2,Oxidation,wet,1140.598752,H2O,31.37,0.215,76,711.632,13,32,01-03-2022,1,Photo,1.70139,13.788,14.820,19.998,202.192,90.000,501.423,4010.176,5092.407,5.050,94.723,29.988,1,photo_li,57.370,H,405,522.143,109.913,2,Etching,219.0,1451.0,3637.0,5729.0,72.983,50.787,1.038,2,Implantation,1.670000e+16,7.880000e+16,7.260000e+17,3.020000e+17,6.000000e+17,31986.009,101.419,882.0,155,79.0,none,[[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 ...,2112,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,NOLSM8483211,3,Oxidation,wet,1275.182502,H2O,45.10,0.214,21,715.498,32,11,28-02-2022,3,Photo,0.89644,16.029,15.005,19.998,204.429,90.000,504.429,4088.587,5110.734,5.114,93.622,29.996,3,photo_li,32.525,I,365,468.930,106.744,3,Etching,177.0,1510.0,3655.0,5726.0,71.550,50.601,1.015,3,Implantation,1.050000e+16,9.510000e+16,6.270000e+17,3.010000e+17,6.000000e+17,32248.621,101.503,868.0,152,78.0,none,[[0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 0 0 0 0 0 0 ...,3333,0
1700,NOLSM8493212,1,Oxidation,wet,1268.105427,H2O,45.07,0.215,22,707.179,32,12,28-02-2022,1,Photo,1.02384,17.587,14.949,20.000,204.266,90.000,504.266,4085.330,5106.662,5.083,93.227,29.994,1,photo_li,28.001,I,365,469.586,106.174,3,Etching,180.0,1433.0,3643.0,5690.0,70.718,50.424,1.011,3,Implantation,6.909170e+15,5.430000e+16,1.870000e+17,3.010000e+17,6.000000e+17,32241.426,101.061,867.

In [41]:
# 이상치 제거
# 0 값이 있는 행의 개수 출력
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
numeric_columns = numeric_columns.drop('Defect')
zero_rows = (df[numeric_columns] <= 0).any(axis=1).sum()

print(zero_rows)

df = df.drop(df[(df[numeric_columns] <= 0).any(axis=1)].index)

39


In [42]:
df.shape

(1654, 59)

In [43]:
selected_columns = ['type','Temp_OXid', 'Vapor', 'ppm','Pressure', 'Oxid_time', 
                   'N2_HMDS', 'pressure_HMDS', 'temp_HMDS', 'temp_HMDS_bake', 'time_HMDS_bake', 'spin1', 'spin2', 'spin3','photoresist_bake','temp_softbake', 'time_softbake',
                    'UV_type', 'Wavelength','Energy_Exposure',
                    'Source_Power','Temp_Etching', 
                    'input_Energy','Temp_implantation','Furance_Temp', 'RTA_Temp',
                     'Defect'                   
                   ]
selected_data = df[selected_columns]

In [44]:
selected_data

,type,Temp_OXid,Vapor,ppm,Pressure,Oxid_time,N2_HMDS,pressure_HMDS,temp_HMDS,temp_HMDS_bake,time_HMDS_bake,spin1,spin2,spin3,photoresist_bake,temp_softbake,time_softbake,UV_type,Wavelength,Energy_Exposure,Source_Power,Temp_Etching,input_Energy,Temp_implantation,Furance_Temp,RTA_Temp,Defect
0,dry,1214.307948,O2,26.69,0.203,120,17.367,15.006,20.001,200.551,90.001,502.391,4017.144,4903.611,5.020,91.918,30.005,I,365,109.628,52.103,72.651,30795.934,103.433,854.0,154,0
1,dry,978.110792,O2,30.86,0.217,137,16.090,15.059,19.993,199.835,89.999,507.183,4072.913,4979.082,4.937,90.976,30.005,H,405,104.917,52.004,70.267,32135.618,105.552,895.0,156,0
2,dry,1176.019149,O2,31.07,0.218,116,19.026,14.930,19.995,202.118,90.001,501.286,4084.939,5031.989,4.888,91.969,30.006,I,365,106.632,50.846,71.231,31057.895,102.598,898.0,152,0
3,dry,933.442733,O2,31.04,0.196,143,18.891,14.947,20.004,194.668,90.000,503.396,4045.078,4969.316,4.933,91.458,30.001,G,436,108.785,51.665,71.295,32140.566,102.963,879.0,155,0
4,wet,1140.598752,H2O,31.37,0.215,76,13.788,14.820,19.998,202.192,90.000,501.423,4010.176,5092.407,5.050,94.723,29.988,H,405,109.913,50.787,72.983,31986.009,101.419,882.0,155,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,wet,1275.182502,H2O,45.10,0.214,21,16.029,15.005,19.998,204.429,90.000,504.429,4088.587,5110.734,5.114,93.622,29.996,I,365,106.744,50.601,71.550,32248.621,101.503,868.0,152,0
1700,wet,1268.105427,H2O,45.07,0.215,22,17.587,14.949,20.000,204.266,90.000,504.266,4085.330,5106.662,5.083,93.227,29.994,I,365,106.174,50.424,70.718,32241.426,101.061,867.0,152,0
1701,wet,1279.160563,H2O,45.12,0.214,21,16.445,14.978,19.994,203.401,90.000,503.401,4068.014,5085.017,5.098,93.547,29.992,I,365,106.108,50.602,71.295,32245.574,101.506,868.0,152,0
1702,wet,1272.804948,H2O,45.34,0.213,20,15.306,15.050,20.002,203.670,90.000,503.670,4073.408,5091.760,5.107,93.487,30.007,I,365,106.765,50.544,71.199,32259.412,101.360,868.0,152,0


In [45]:
selected_data_dummy = pd.get_dummies(selected_data, drop_first = False)
selected_data_dummy.head()

,Temp_OXid,ppm,Pressure,Oxid_time,N2_HMDS,pressure_HMDS,temp_HMDS,temp_HMDS_bake,time_HMDS_bake,spin1,spin2,spin3,photoresist_bake,temp_softbake,time_softbake,Wavelength,Energy_Exposure,Source_Power,Temp_Etching,input_Energy,Temp_implantation,Furance_Temp,RTA_Temp,Defect,type_dry,type_wet,Vapor_H2O,Vapor_O2,UV_type_G,UV_type_H,UV_type_I
0,1214.307948,26.69,0.203,120,17.367,15.006,20.001,200.551,90.001,502.391,4017.144,4903.611,5.020,91.918,30.005,365,109.628,52.103,72.651,30795.934,103.433,854.0,154,0,1,0,0,1,0,0,1
1,978.110792,30.86,0.217,137,16.090,15.059,19.993,199.835,89.999,507.183,4072.913,4979.082,4.937,90.976,30.005,405,104.917,52.004,70.267,32135.618,105.552,895.0,156,0,1,0,0,1,0,1,0
2,1176.019149,31.07,0.218,116,19.026,14.930,19.995,202.118,90.001,501.286,4084.939,5031.989,4.888,91.969,30.006,365,106.632,50.846,71.231,31057.895,102.598,898.0,152,0,1,0,0,1,0,0,1
3,933.442733,31.04,0.196,143,18.891,14.947,20.004,194.668,90.000,503.396,4045.078,4969.316,4.933,91.458,30.001,436,108.785,51.665,71.295,32140.566,102.963,879.0,155,0,1,0,0,1,1,0,0
4,1140.598752,31.37,0.215,76,13.788,14.820,19.998,202.192,90.000,501.423,4010.176,5092.407,5.050,94.723,29.988,405,109.913,50.787,72.983,31986.009,101.419,882.0,155,0,0,1,1,0,0,1,0


In [46]:
# 훈련 데이터와 검정 데이터를 7:3 비율로 분할
df_train, df_test = train_test_split(selected_data_dummy, test_size = 0.3 , random_state = 7777)
print("train data size : {}".format(df_train.shape))
print("test data size : {}".format(df_test.shape))

train data size : (1157, 31)
test data size : (497, 31)


In [47]:
# 로지스틱 모델 생성
log_model = Logit.from_formula("Defect ~ Temp_OXid + ppm + Pressure + Oxid_time + N2_HMDS + pressure_HMDS \
+ temp_HMDS + temp_HMDS_bake + time_HMDS_bake + spin1 + spin2 + spin3 + photoresist_bake + temp_softbake \
+ time_softbake + Wavelength + Energy_Exposure + Source_Power + Temp_Etching + input_Energy + \
Temp_implantation + Furance_Temp + RTA_Temp + type_dry + type_wet + \
Vapor_H2O + Vapor_O2 + UV_type_G + UV_type_H + UV_type_I", df_train)

log_result = log_model.fit()
print(log_result.summary())

         Current function value: 0.231854
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                 Defect   No. Observations:                 1157
Model:                          Logit   Df Residuals:                     1131
Method:                           MLE   Df Model:                           25
Date:                Tue, 30 May 2023   Pseudo R-squ.:                  0.1093
Time:                        11:29:12   Log-Likelihood:                -268.26
converged:                      False   LL-Null:                       -301.19
Covariance Type:            nonrobust   LLR p-value:                 1.554e-05
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           269.2962        nan        nan        nan         nan         nan
Temp_OXid            -0.0020      0.001     

예측

In [48]:
# train 데이터 예측
y_pred_train = log_result.predict(df_train)

y_pred_train_class = (y_pred_train > 0.5).astype(int)
print("Train 예측 결과 \n", y_pred_train_class.head(), '\n')
print("Confusion Matrix: \n{}".format(confusion_matrix(df_train["Defect"], y_pred_train_class)), "\n")

# test 데이터 예측
y_pred_test = log_result.predict(df_test)

y_pred_test_class = (y_pred_test > 0.5).astype(int)
print("Test 예측 결과 \n", y_pred_test_class.head(), "\n")
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["Defect"], y_pred_test_class)), "\n")

Train 예측 결과 
 209    0
387    0
648    0
897    0
838    0
dtype: int64 

Confusion Matrix: 
[[1073    0]
 [  84    0]] 

Test 예측 결과 
 407     0
74      0
552     0
365     0
1157    0
dtype: int64 

Confusion Matrix: 
[[463   0]
 [ 34   0]] 



In [49]:
# 실제 train 데이터와 예측결과 비교
print('Train 예측/분류 결과')
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_train["Defect"], y_pred_train_class)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_train["Defect"], y_pred_train_class)), "\n")
print(classification_report(df_train["Defect"], y_pred_train_class, digits = 3))
      
# 실제 test 데이터와 예측 결과 비교
print('Test 예측/분류 결과')
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_test["Defect"], y_pred_test_class)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["Defect"], y_pred_test_class)), "\n")
print(classification_report(df_test["Defect"], y_pred_test_class, digits = 3))

Train 예측/분류 결과
Accuracy: 0.927

Confusion Matrix: 
[[1073    0]
 [  84    0]] 

              precision    recall  f1-score   support

           0      0.927     1.000     0.962      1073
           1      0.000     0.000     0.000        84

    accuracy                          0.927      1157
   macro avg      0.464     0.500     0.481      1157
weighted avg      0.860     0.927     0.892      1157

Test 예측/분류 결과
Accuracy: 0.932

Confusion Matrix: 
[[463   0]
 [ 34   0]] 

              precision    recall  f1-score   support

           0      0.932     1.000     0.965       463
           1      0.000     0.000     0.000        34

    accuracy                          0.932       497
   macro avg      0.466     0.500     0.482       497
weighted avg      0.868     0.932     0.899       497



# SMOTE

In [50]:
# 설명변수, 목표변수 데이터 구분
df_raw_x = selected_data_dummy.drop('Defect', axis=1, inplace=False)
df_raw_y = selected_data_dummy['Defect']

In [51]:
# 데이터 분할
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_x, df_raw_y, 
                                                                test_size=0.3, random_state = 77)

print("분할 전 설명변수 데이터:", df_raw_x.shape)
print("분할 후 설명변수 데이터:Train", df_train_x.shape, " Test", df_test_x.shape)

분할 전 설명변수 데이터: (1654, 30)
분할 후 설명변수 데이터:Train (1157, 30)  Test (497, 30)


In [52]:
# Over-sampling : SMOTE
from imblearn.over_sampling import SMOTE

# 목표변수 빈도 확인
print(selected_data_dummy.value_counts(['Defect']), "\n")
print("Target=1 비율 ", selected_data_dummy.value_counts(selected_data_dummy['Defect']==1)/len(selected_data_dummy))

Defect
0         1536
1          118
dtype: int64 

Target=1 비율  Defect
False    0.928658
True     0.071342
dtype: float64


In [53]:
# Over-sampling 설정
sm = SMOTE(sampling_strategy='auto')

# train 데이터를 이용한 Over-sampling
x_resampled, y_resampled = sm.fit_resample(df_train_x, df_train_y)

# 결과확인
print('Over-Sampling 전:\n', df_train_y.value_counts(), "\n")
print('Over-Sampling 후 Train X: {}'.format(x_resampled.shape))
print('Over-Sampling 후 Train Y: {} \n'.format(y_resampled.shape))
print("Over-Sampling 후 '1':{}".format(sum(y_resampled==1)))
print("Over-Sampling 후 '0':{}".format(sum(y_resampled==0)))

Over-Sampling 전:
 0    1072
1      85
Name: Defect, dtype: int64 

Over-Sampling 후 Train X: (2144, 30)
Over-Sampling 후 Train Y: (2144,) 

Over-Sampling 후 '1':1072
Over-Sampling 후 '0':1072


In [54]:
# 데이터 결합
df_resampled = pd.concat([x_resampled, y_resampled], axis=1)
print(df_resampled.head())

     Temp_OXid    ppm  Pressure  Oxid_time  N2_HMDS  pressure_HMDS  temp_HMDS  \
0  1097.859817  34.48     0.219        111   18.899         15.013     20.001   
1  1268.370063  44.55     0.215         22   17.218         15.146     20.001   
2   874.398039  22.10     0.219        226   16.706         14.957     20.004   
3  1198.323089  26.69     0.203        120   14.060         15.006     20.001   
4   891.039700  22.17     0.219         25   16.090         14.908     20.003   

   temp_HMDS_bake  time_HMDS_bake    spin1     spin2     spin3  \
0         200.943          89.999  504.074  4076.151  5036.475   
1         198.569          90.000  498.569  3971.388  4964.235   
2         204.484          90.000  501.388  4027.760  5034.700   
3         200.551          90.001  503.630  4010.591  4903.611   
4         204.234          90.000  504.234  4084.680  5105.850   

   photoresist_bake  temp_softbake  time_softbake  Wavelength  \
0             5.034         90.139         29.998  

In [58]:
# 로지스틱 모델 생성
log_result_SMOTE = LogisticRegression(random_state=77)
log_result_SMOTE.fit(x_resampled, y_resampled)

# train 모델 정확도
print("Accuracy on training set: {:.3f}".format(log_result_SMOTE.score(x_resampled, y_resampled)))
# test 모델 정확도
print("Accuracy on test set: {:.3f}".format(log_result_SMOTE.score(df_test_x, df_test_y)))

Accuracy on training set: 0.617
Accuracy on test set: 0.584


In [56]:
y_pred_train = log_result_SMOTE.predict(x_resampled)

train_ConMat = confusion_matrix(y_resampled, y_pred_train)
print("Confusion Matrix:")
print(train_ConMat)

Confusion Matrix:
[[638 434]
 [387 685]]


In [57]:
# test set 예측
y_pred_test = log_result_SMOTE.predict(df_test_x)

# confusion matrix 계산
test_ConMat = confusion_matrix(df_test_y, y_pred_test)
print("Confusion Matrix:")
print(test_ConMat)

Confusion Matrix:
[[269 195]
 [ 12  21]]


In [60]:
# train 데이터 예측
y_pred_train = log_result_SMOTE.predict(x_resampled)

# y_pred_train_class = (y_pred_train > 0.5).astype(int)
# print("Train 예측 결과 \n", y_pred_train_class.head(), '\n')
print("Confusion Matrix: \n{}".format(confusion_matrix(y_resampled, y_pred_train)), "\n")

# test 데이터 예측
y_pred_test = log_result_SMOTE.predict(df_test_x)

# y_pred_test_class = (y_pred_test > 0.5).astype(int)
# print("Test 예측 결과 \n", y_pred_test_class.head(), "\n")
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test_y, y_pred_test)), "\n")

Confusion Matrix: 
[[638 434]
 [387 685]] 

Confusion Matrix: 
[[269 195]
 [ 12  21]] 



In [61]:
# 실제 train 데이터와 예측결과 비교
print('Train 예측/분류 결과')
print("Accuracy: {0:.3f}\n".format(accuracy_score(y_resampled, y_pred_train)))
print("Confusion Matrix: \n{}".format(confusion_matrix(y_resampled, y_pred_train)), "\n")
print(classification_report(y_resampled, y_pred_train, digits = 3))
      
# 실제 test 데이터와 예측 결과 비교
print('Test 예측/분류 결과')
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_test_y, y_pred_test)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test_y, y_pred_test)), "\n")
print(classification_report(df_test_y, y_pred_test, digits = 3))

Train 예측/분류 결과
Accuracy: 0.617

Confusion Matrix: 
[[638 434]
 [387 685]] 

              precision    recall  f1-score   support

           0      0.622     0.595     0.608      1072
           1      0.612     0.639     0.625      1072

    accuracy                          0.617      2144
   macro avg      0.617     0.617     0.617      2144
weighted avg      0.617     0.617     0.617      2144

Test 예측/분류 결과
Accuracy: 0.584

Confusion Matrix: 
[[269 195]
 [ 12  21]] 

              precision    recall  f1-score   support

           0      0.957     0.580     0.722       464
           1      0.097     0.636     0.169        33

    accuracy                          0.584       497
   macro avg      0.527     0.608     0.445       497
weighted avg      0.900     0.584     0.685       497

